In [25]:
from itertools import permutations, tee

from IPython.display import display_latex, Latex

In [69]:
def pairs(xs):
    return ((x, y) for i, x in enumerate(xs) for y in xs[i+1:])

def parity(xs):
    return sum((x > y for x, y in pairs(xs))) % 2

In [70]:
l = [1, 2, 3]
ps = list(permutations(l))

for p in ps:
    print(p, parity(p))

(1, 2, 3) 0
(1, 3, 2) 1
(2, 1, 3) 1
(2, 3, 1) 0
(3, 1, 2) 0
(3, 2, 1) 1


In [5]:
n = 3
for p in ps:
    ss = []
    for k in range(0, n):
        i_k = p[k]
        i_ls = []
        for l in range(0, k):
            i_l = p[l]
            if i_k < i_l:
                i_ls.append(r"\left| \phi_%d \right|" % i_l)
        if len(i_ls) == 0:
            continue
        elif len(i_ls) == 1:
            s = r"\left| \phi_%d \right| %s" % (i_k, i_ls[0])
        else:
            args = (i_k, ' + '.join(i_ls))
            s = r"\left| \phi_%d \right| \left( %s \right)" % args
        ss.append(s)
    s = " + ".join(ss) if ss else '0'
    display_latex(Latex(r"\[ %s \rightarrow %s \]" % (p, s)))

In [6]:
def ternary_to_bin(*ternary):
    lhs = r"\left[{}, {}, {}\right]".format(*ternary)
    
    r1 = "{1} \circ [{2}, {3}]"
    r2 = r"(-1)^{{ \left|{1}\right|\left|{2}\right| }} {2} \circ [{1}, {3}]"
    r3 = r"(-1)^{{ \left|{1}\right|\left|{3}\right|+\left|{2}\right|\left|{3}\right| }} {3} \circ [{1}, {2}]"
    r = "{} - {} + {}".format(r1, r2, r3)
    rhs = r.format(None, *ternary)

    latex = r"\[ {} = {} \]".format(lhs, rhs)
    display_latex(Latex(latex))

ternary_to_bin(r"\phi_1 \circ [\psi_1, \psi_2]", r"\phi_2", r"\phi_3")
ternary_to_bin(r"\phi_1 \circ [\phi_2, \phi_3]", r"\phi_4", r"\phi_5")
ternary_to_bin(r"\phi_1" , r"\phi_2", r"\phi_3")


In [127]:
def comm(indexes, only_starts_with=None, display=True):
    def perm_to_power(p):
        def sort(x, y): return (x, y) if x < y else (y, x)
        parity_s = "1" if parity(p) > 0 else ""
        products = [sort(el, other) for i, el in enumerate(p) for other in p[i+1:] if el > other]
        sum_s = " + ".join([r"|\phi_%d||\phi_%d|" % p for p in sorted(products)])
        if parity_s and sum_s:
            return "%s + %s" % (parity_s, sum_s)
        return parity_s or sum_s

    def perm_to_term(p):
        power = perm_to_power(p)
        sign = r"(-1)^{%s}" % power if power else ""
        return sign + r" \circ ".join((r"\phi_%d" % d for d in p))

    perms = sorted(permutations(indexes), key=lambda xs: xs[0])
    if only_starts_with:
        perms = [p for p in perms if p[0] == only_starts_with]
    terms = map(perm_to_term, perms)
    left = "[%s]" % ", ".join((r"\phi_%d" % i for i in indexes))
    right = r"& + \\".join(terms)
    latex = r"\begin{align*} %s& = \\ %s \end{align*}" % (left, right)
    if display:
        display_latex(Latex(latex))
    return latex

In [131]:
l1 = comm([1,2,3,4], only_starts_with=1)
l2 = comm([2,3,4])

latex = r"""
\begin{tabular}{ l l }
  %s & %s
\end{tabular}
""" % (l1, l2)

print(latex)
display_latex(Latex(latex))


\begin{tabular}{ l l }
  \begin{align*} [\phi_1, \phi_2, \phi_3, \phi_4]& = \\ \phi_1 \circ \phi_2 \circ \phi_3 \circ \phi_4& + \\(-1)^{1 + |\phi_3||\phi_4|}\phi_1 \circ \phi_2 \circ \phi_4 \circ \phi_3& + \\(-1)^{1 + |\phi_2||\phi_3|}\phi_1 \circ \phi_3 \circ \phi_2 \circ \phi_4& + \\(-1)^{|\phi_2||\phi_3| + |\phi_2||\phi_4|}\phi_1 \circ \phi_3 \circ \phi_4 \circ \phi_2& + \\(-1)^{|\phi_2||\phi_4| + |\phi_3||\phi_4|}\phi_1 \circ \phi_4 \circ \phi_2 \circ \phi_3& + \\(-1)^{1 + |\phi_2||\phi_3| + |\phi_2||\phi_4| + |\phi_3||\phi_4|}\phi_1 \circ \phi_4 \circ \phi_3 \circ \phi_2 \end{align*} & \begin{align*} [\phi_2, \phi_3, \phi_4]& = \\ \phi_2 \circ \phi_3 \circ \phi_4& + \\(-1)^{1 + |\phi_3||\phi_4|}\phi_2 \circ \phi_4 \circ \phi_3& + \\(-1)^{1 + |\phi_2||\phi_3|}\phi_3 \circ \phi_2 \circ \phi_4& + \\(-1)^{|\phi_2||\phi_3| + |\phi_2||\phi_4|}\phi_3 \circ \phi_4 \circ \phi_2& + \\(-1)^{|\phi_2||\phi_4| + |\phi_3||\phi_4|}\phi_4 \circ \phi_2 \circ \phi_3& + \\(-1)^{1 + |\phi_2||\phi_3| 